<a href="https://colab.research.google.com/github/tonystz/gitpod/blob/main/pycuda_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to CUDA and PyCUDA

In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
import numpy as np
import pandas as pd


df=pd.read_csv('drive/MyDrive/a.csv').dropna()
print(df)
a= df.to_numpy()
print(a)
print(a.shape,a.size,a.dtype)

# python to check risk IP, check threshhold: >5 times within 6 minute

In [5]:
from datetime import datetime
import csv
import socket, struct
import numpy as np

ip2long=lambda ip:struct.unpack("!L", socket.inet_aton(ip))[0]
long2ip=lambda long:socket.inet_ntoa(struct.pack('!L', long))
date2int=lambda current_date: current_date.year*10000000000 + current_date.month * 100000000 +  current_date.day * 1000000 + current_date.hour*10000 + current_date.minute*100 + current_date.second

hack_type={
    'Ftp访问日志':0,
    'SQL 注入':1,
    'Web Shell攻击':2,
    '暴力破解':3,
    '端口扫描':4,
    '缓冲区溢出攻击':5,
    '弱口令漏洞':6
}

numdata=[]
with open('drive/MyDrive/a.csv', newline='') as csvfile:
    data = csv.reader(csvfile)
    next(data) #skip header
    for row in data:
        if len(row[0]) == 0: continue
        numdata.append([date2int(datetime.strptime(row[1]+' '+row[2],'%Y/%m/%d %H:%M:%S')),hack_type.get(row[3],-1),ip2long(row[4])])
        print(row)
print(numdata)
a=np.array(numdata,dtype=np.uint64)

# r=np.zeros(a.shape,dtype=a.dtype)

rdict={}
for i in a:
    if i[2] in rdict:
        rdict[i[2]][0] +=1
        rdict[i[2]][1].append(i[0])
    else:
        rdict[i[2]]=[1,[i[0]]]

# print(rdict)

# check threshhold: >5 within 6 minute
th_count=5
th_min=6*100

for k,v in rdict.items():
    if v[0] > th_count:
        avg_time=int(sum(v[1])/len(v[1]))
        if [i for i in v[1] if i < avg_time + th_min]:
            print(long2ip(k))

['1', '2023/3/3', '9:31:01', 'SQL 注入', '192.168.10.101', '32546', '172.10.166.204', '3306', 'Huang.XD', 'Failed']
['2', '2023/3/3', '9:31:01', '端口扫描', '192.168.10.102', '32547', '172.10.166.205', '443', 'Huang.XD', 'Failed']
['3', '2023/3/3', '9:31:01', '暴力破解', '192.168.10.103', '32548', '172.10.166.206', '22', 'Huang.XD', 'Failed']
['4', '2023/3/3', '9:31:01', 'Web Shell攻击', '192.168.10.104', '32549', '172.10.166.207', '8080', 'Liu.YD', 'Failed']
['5', '2023/3/3', '9:31:01', 'Ftp访问日志', '192.168.10.105', '32550', '172.10.166.208', '21', 'Fang.ZL', 'Failed']
['6', '2023/3/3', '9:31:01', '弱口令漏洞', '192.168.10.106', '32551', '172.10.166.209', '143', 'Huang.XD', 'Failed']
['7', '2023/3/3', '9:31:01', 'SQL 注入', '192.168.10.107', '32552', '172.10.166.210', '3306', 'Liu.YD', 'Failed']
['8', '2023/3/3', '9:31:01', '端口扫描', '192.168.10.108', '32553', '172.10.166.211', '443', 'Fang.ZL', 'Failed']
['9', '2023/3/3', '9:31:01', '暴力破解', '192.168.10.101', '32554', '172.10.166.212', '22', 'Chen.JT', 'Fa

In [ ]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [ ]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

In [ ]:
b = numpy.random.randn(4,4)
b = b.astype(numpy.float32)
c = numpy.random.randn(4,4)
c = c.astype(numpy.float32)

In [ ]:
mod2 = SourceModule("""
  __global__ void add2(float *a, float *b)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] += b[idx];
  }
  """)

In [ ]:
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)


In [ ]:
func = mod2.get_function("add2")
func(b_gpu,c_gpu, block=(4,4,1))

In [ ]:
added = numpy.empty_like(b)
cuda.memcpy_dtoh(added, b_gpu)
print(added)
print(b)
print(c)

# Exercises

1. Write a cuda kernel to find the elementwise square of a matrix
2. Write a cuda kernel to find a matrix, which when added to the given matrix results in every element being equal to zero
3. Write a cuda kernel to multiply two matrices:
    1. Assume square matrices, with dimensions < 1024
    2. Assume square matrices, with dimensions > 1024
    3. Assume non-square matrices, with dimensions > 1024

In [ ]:
1. 